# Proyecto 200Eng - Black list

Transferencia a servidor



### Importa dependencias

In [1]:
# pip install pymysql

In [2]:
#importa dependencias
from pymysql import connect
import pandas as pd
from datetime import datetime, date, time, timedelta
from time import sleep
from textwrap import dedent
import matplotlib.pyplot as plt
import seaborn as sns
from millify import millify, prettify
import os
from sqlalchemy import create_engine
import plotly.express as px
import random


In [3]:
round(random.random(), 2)

0.76

In [4]:
datetime.now()

datetime.datetime(2024, 9, 23, 12, 46, 36, 947630)

In [5]:
datetime.now().hour

12

In [6]:
date(2024, 8, 18) == datetime(2024, 8, 18, 0, 0, 0)

False

In [7]:
datetime.now() - timedelta(days=30)

datetime.datetime(2024, 8, 24, 12, 46, 37, 450051)

In [8]:
start = datetime.now()
sleep(3)
end = datetime.now()
elapsed = end - start
print(str(elapsed))

0:00:03.005553


### Conexión con todos los marcadores

In [9]:
# lista de marcadores
url_marcadores = ['83.138.55.125',
              '193.219.97.92',
              '193.219.97.90',
              '83.138.55.115',
              '94.177.9.69',
              '83.138.55.163',
              ]

ind_marcadores = [1, 2, 3, 4, 6, 7]

In [10]:
user = 'carlosai'
password = 'OKJmBUIFnh'
port = '3306'
database = 'mbilling'

In [11]:
# creación de un diccionario de bd para cada marcadro con pymysql

def connect_to_db():
    bd = {}
    cursores = {}

    for i, ind in enumerate(ind_marcadores):
        # crea la conexión
        bd[f'marcador_{ind}'] = connect(host = url_marcadores[i],
                                        user=user,
                                        passwd=password,
                                        database=database,
                                        )

        # crea el cursor
        cursores[f'cursor_marcador_{ind}'] = bd[f'marcador_{ind}'].cursor()
    return bd, cursores

bd, cursores = connect_to_db()

bd


{'marcador_1': <pymysql.connections.Connection at 0x131a373b0>,
 'marcador_2': <pymysql.connections.Connection at 0x13089c9e0>,
 'marcador_3': <pymysql.connections.Connection at 0x131acf770>,
 'marcador_4': <pymysql.connections.Connection at 0x131acf260>,
 'marcador_6': <pymysql.connections.Connection at 0x131acf9b0>,
 'marcador_7': <pymysql.connections.Connection at 0x131acf5f0>}

In [12]:
# creación de un diccionario de bd para cada marcadro con sqlalchemy

bd_sqlalch = {}

for i, ind in enumerate(ind_marcadores):
    # crea la conexión
    cnx = create_engine(f'mysql+pymysql://{user}:{password}@{url_marcadores[i]}:{port}/{database}')
    bd_sqlalch[f'marcador_{ind}'] = cnx.connect()


In [13]:
bd

{'marcador_1': <pymysql.connections.Connection at 0x131a373b0>,
 'marcador_2': <pymysql.connections.Connection at 0x13089c9e0>,
 'marcador_3': <pymysql.connections.Connection at 0x131acf770>,
 'marcador_4': <pymysql.connections.Connection at 0x131acf260>,
 'marcador_6': <pymysql.connections.Connection at 0x131acf9b0>,
 'marcador_7': <pymysql.connections.Connection at 0x131acf5f0>}

In [14]:
bd_sqlalch

{'marcador_1': <sqlalchemy.engine.base.Connection at 0x131b2c1d0>,
 'marcador_2': <sqlalchemy.engine.base.Connection at 0x131a34ef0>,
 'marcador_3': <sqlalchemy.engine.base.Connection at 0x10fd6b110>,
 'marcador_4': <sqlalchemy.engine.base.Connection at 0x131beb380>,
 'marcador_6': <sqlalchemy.engine.base.Connection at 0x10fd6b170>,
 'marcador_7': <sqlalchemy.engine.base.Connection at 0x131bebf50>}

In [15]:
cursores

{'cursor_marcador_1': <pymysql.cursors.Cursor at 0x1308c58b0>,
 'cursor_marcador_2': <pymysql.cursors.Cursor at 0x131acf710>,
 'cursor_marcador_3': <pymysql.cursors.Cursor at 0x131acf800>,
 'cursor_marcador_4': <pymysql.cursors.Cursor at 0x131acee70>,
 'cursor_marcador_6': <pymysql.cursors.Cursor at 0x131acf950>,
 'cursor_marcador_7': <pymysql.cursors.Cursor at 0x131acf530>}

### Extracción de datos

In [16]:
# rango de 1 mes
fecha = datetime(2023, 9, 30)
dias = 15
fecha_menos_30 = (fecha - timedelta(days=dias)).strftime('%Y-%m-%d')
start = '2023-01-12'

if start > fecha_menos_30:
    fecha_inicio = start
else:
    fecha_inicio = fecha_menos_30

fecha = fecha.strftime('%Y-%m-%d')

print(f'Fecha de inicio: {fecha_inicio}\nFecha de final: {fecha}')


Fecha de inicio: 2023-09-15
Fecha de final: 2023-09-30


In [17]:
# define la consulta (query)
query = dedent(f"""
        SELECT starttime, calledstation, real_sessiontime
        FROM pkg_cdr
        WHERE starttime BETWEEN '{fecha_inicio}' AND '{fecha}';
        """)

print(f'Query a correr:\n{query}')


Query a correr:

SELECT starttime, calledstation, real_sessiontime
FROM pkg_cdr
WHERE starttime BETWEEN '2023-09-15' AND '2023-09-30';



In [18]:
# crea una función para devolver cada df con pymysql
def get_df(bd, bd_name):
    df_temp = pd.read_sql(query, bd)
    df_temp['bd'] = bd_name

    return df_temp

In [19]:
# crea una función para devolver cada df con sqlalchemy
def get_df_sqlalch(bd, bd_name):
    sql_query = pd.read_sql_query(query, bd)
    df_temp = pd.DataFrame(sql_query)
    df_temp['bd'] = bd_name

    return df_temp

In [20]:
#lista de df
# df_list = [get_df(bd[f'marcador_{ind}'], f'marcador_{ind}') for ind in ind_marcadores]

df_list = []
for ind in ind_marcadores:
    query_ok = False
    conn_ok = False
    print(f'Consultando marcador_{ind}...')
    while not query_ok:
        try:
            df_list.append(get_df(bd[f'marcador_{ind}'], f'marcador_{ind}'))
            query_ok = True
            print(f'Consulta marcador_{ind} OK')
        except:
            print(f'Consulta marcador_{ind} con ERROR')
            while not conn_ok:
                try:
                    bd, _ = connect_to_db()
                    conn_ok = True
                    print(f'Nueva conexión bd OK')
                except:
                    print(f'Nueva conexión bd con ERROR')
                    sleep(5)
        


Consultando marcador_1...


/var/folders/4x/0wzldd8s3md_vc0bvz7ndzth0000gn/T/ipykernel_11218/2795151752.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_temp = pd.read_sql(query, bd)


Consulta marcador_1 OK
Consultando marcador_2...


/var/folders/4x/0wzldd8s3md_vc0bvz7ndzth0000gn/T/ipykernel_11218/2795151752.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_temp = pd.read_sql(query, bd)


In [ ]:
ind_marcadores

In [ ]:
len(df_list)

In [ ]:
df_list[2]

In [ ]:
df = pd.concat(df_list, axis=0)
df

In [ ]:
df.info()

In [ ]:
df['weekday'] = df.starttime.dt.weekday
df['hour'] = df.starttime.dt.hour
df

In [ ]:
df = df.set_index('starttime')
df = df.sort_index()
df

In [ ]:
df.memory_usage(deep=True)

In [ ]:
df.info()

In [36]:
df.calledstation = df.calledstation.astype('category')
df.real_sessiontime = df.real_sessiontime.astype('int')
df.bd = df.bd.astype('category')

In [ ]:
df.info()

In [ ]:
df.memory_usage(deep=True)

In [ ]:
df.index.nunique()

In [ ]:
df.shape

In [ ]:
df.info()

Observaciones
* La columna `callerid` es igual a la columna `calleddestination`

In [ ]:
df.nunique().sort_values(ascending=False)

In [ ]:
df_dest_calls = df.calledstation.value_counts().to_frame()
df_dest_calls = df_dest_calls.rename(columns={'count': 'num_llamadas'})
df_dest_calls

### Análisis de información

In [ ]:
df_dest = df.groupby(['calledstation']).real_sessiontime.sum().sort_values(ascending=False).to_frame()
df_dest

In [ ]:
df_dest = df_dest.join(df_dest_calls, on='calledstation')
df_dest

In [ ]:
df_dest['dur_prom'] = df_dest.real_sessiontime / df_dest.num_llamadas
df_dest

In [ ]:
df_dest.sort_values(by='num_llamadas', ascending=False)

In [210]:
def P5(x):
    return x.quantile(0.05)

def P25(x):
    return x.quantile(0.25)

def P50(x):
    return x.quantile(0.5)

def P75(x):
    return x.quantile(0.75)

def P95(x):
    return x.quantile(0.95)

In [211]:
# %%time
# df_dest_stats = df.groupby('calledstation').aggregate({'real_sessiontime': ['max', 'min', P25, P50, P75]})


In [212]:
# df_dest_stats

In [213]:
# df_dest_stats.columns = df_dest_stats.columns.map('_'.join).str.strip('_')
# df_dest_stats

In [214]:
# df_dest = df_dest.join(df_dest_stats, on='calledstation')
# df_dest

In [215]:
def ratio(df_func, last_n=2):
    df_temp = df_func.tail(last_n)
    ratio = df_temp['real_sessiontime'].std() / df_temp['real_sessiontime'].mean()
    return ratio

In [ ]:
df_dest_ratio = df.groupby('calledstation').apply(ratio).to_frame()
df_dest_ratio = df_dest_ratio.rename(columns={0:'flat_ratio'})
df_dest_ratio

In [ ]:
df_dest = df_dest.join(df_dest_ratio, on='calledstation')
df_dest

In [ ]:
umbral = 0.005
df_dest_flat = df_dest[df_dest.flat_ratio <= umbral].sort_values(by='num_llamadas', ascending=False)
df_dest_flat

In [ ]:
index_lm = df_dest_flat.index
index_lm

In [ ]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
block_numbers = ''
ind = 0
num_block = 10


while ind < num_block:
    block_numbers += "("
    block_numbers += index_lm[ind]
    block_numbers += ","
    block_numbers += f"'Número agregado por Carlos Gorricho desde script the Python el {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}'"
    if ind < num_block - 1:
        block_numbers += "),\n"
    else:
        block_numbers += ")"

    ind += 1


print(block_numbers)


In [ ]:
# define la consulta (query)
add_query = dedent(f"""INSERT INTO `pkg_campaign_restrict_phone` (number, description) VALUES {block_numbers};""")

print(f'Query a correr:\n{add_query}')


In [ ]:
# ejecuta consulta en todos los marcadores
for key in cursores:
    cursores[key].execute(add_query)
    print(f'Ejectando consulta en {key} - Agregadas {cursores[key].rowcount} filas')
    


In [ ]:
# compromete los cambios en todos los marcadores
for base in bd:
    bd[base].commit()
    print(f'Condirmando cambios en {base}')
    

### Visualizaciones

#### Flatliners

In [ ]:
df

In [ ]:
index_lm = df_dest_flat.index
index_lm

In [ ]:
df_dest_flat.flat_ratio.mean()

In [ ]:
rows=5
cols=5
indice=0
i=0
index_lm[i+indice*rows*cols]

In [ ]:
df_dest.index[i+indice*rows*cols]

In [251]:
def plot_flat_ts(df, ind=0):    
    rows = 5
    cols = 5
    indice = ind
    fix, ax = plt.subplots(rows,cols,
                        figsize=(20,rows * 4),
                        sharey=True,
                        sharex=True,
                        squeeze=False)

    for i in range(rows * cols):
        ax[i//5, i%5].plot(df[df['calledstation']==index_lm[i+indice*rows*cols]].real_sessiontime)
        ax[i//5, i%5].set_title(f"{i+1+indice*rows*cols}. {df_dest_flat.index[i+indice*rows*cols]} ({df_dest_flat.loc[index_lm[i+indice*rows*cols], 'flat_ratio']:.3f})\n")
        ax[i//5, i%5].set_xticklabels([])

    plt.show()


In [ ]:
%%time
indice = 10
plot_flat_ts(df, ind=indice)

In [ ]:
index_lm[:20]

In [ ]:
df_grafica = df[df.calledstation.isin(index_lm[i+indice*rows*cols:i+indice*rows*cols+rows*cols])]
df_grafica

#### Comportamiento de llamadas en el tiempo

In [ ]:
df.info()

In [ ]:
df.groupby('hour').count().calledstation.plot(kind='bar')

In [ ]:
sns.barplot(data=df.groupby('hour').count(),
            x=df.groupby('hour').count().index,
            y='calledstation')

In [ ]:
fig = px.bar(df.groupby('hour').count(), 
             x=df.groupby('hour').count().index, 
             y='calledstation')
fig.show()

In [ ]:
# lunes = 0, domingo = 6
df.groupby('weekday').count().calledstation.plot(kind='bar')

In [ ]:
sns.barplot(data=df.groupby('weekday').count(),
            x=df.groupby('weekday').count().index,
            y='calledstation')

In [ ]:
fig = px.bar(df.groupby('weekday').count(), 
             x=df.groupby('weekday').count().index, 
             y='calledstation')
fig.show()

#### Duración de llamadas en el tiempo

In [ ]:
index_lm

In [ ]:
dest_num = '18003715786'
last_n=10

fix, ax1 = plt.subplots(figsize=(5,5))

plt.xticks(rotation='vertical')

sns.scatterplot(data = df[df.calledstation == dest_num].tail(last_n),
             x = df[df.calledstation == dest_num].tail(last_n).index,
             y = 'real_sessiontime',
             hue = 'bd',
             s=50,                         
             ax=ax1,
             )

ax2 = ax1.twinx()

sns.lineplot(data = df[df.calledstation == dest_num].tail(last_n),
             x = df[df.calledstation == dest_num].tail(last_n).index,
             y = 'real_sessiontime',
             ax=ax2,
             )

plt.title(f'{dest_num}')

plt.show()

In [ ]:
df[df.calledstation == dest_num]

In [ ]:
low = len(df[df.calledstation == dest_num]) - 2
hi = len(df[df.calledstation == dest_num])
df[df.calledstation == dest_num].real_sessiontime.iloc[low:hi].std() / df[df.calledstation == dest_num].real_sessiontime.iloc[low:hi].mean()

#### Dispersión de duración de llamadas según frecuencia

In [ ]:
fix, ax1 = plt.subplots(figsize=(5,5))

plt.xticks(rotation='vertical')

sns.lineplot(data = df_dest,
             x = df_dest.num_llamadas,
             y = df_dest.dur_prom,
             errorbar=('ci', 100),
             ax=ax1,
             )

sns.lineplot(data = df_dest,
             x = df_dest.num_llamadas,
             y = df_dest.dur_prom.mean(),
             )


plt.show()


#### Frecuencia de número de llamadas

In [ ]:
df_dest.num_llamadas.value_counts()

In [ ]:
df_dest.num_llamadas.value_counts().plot(kind='bar')

#### Histogramas

In [ ]:
# por duraciones promedio
df_dest.dur_prom.hist(bins=100)

In [ ]:
# por duraciones reales
df.real_sessiontime.hist(bins=100)